# Planet Download Example
This notebook shows how to download planet imagery uploaded to a private s3 bucket for OPERA DSWx validation

In [ ]:
import geopandas as gpd
import boto3
import os

### AWS Credentials
In order to download imagery from the private bucket, JPL RSA access and OPERA Calval AWS credentials are needed

In [ ]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

### S3 Buckets
A table containing image metadata is located on a public s3 bucket, but the commercial image files can't be openly shared and are stored in a seperate private bucket. S3 keys (path within a bucket) that point to the image file locations are stored in the metadata table in the public bucket

In [ ]:
bucket_name = 'opera-calval-database-dswx'
private_bucket_name = 'opera-calval-database-dswx-private'

### Read Image metadata table
Image metadata is stored in a geojson table, which can be read as a geodataframe using geopandas. This dataframe can be spatial searched or searched by metadata fields

In [ ]:
obj = s3.Object(bucket_name,'image.geojson')
image_table_data = obj.get()['Body']
images = gpd.read_file(image_table_data)
images.head()

### Download Function
This function takes a single row of the image geodataframe and a download location (folder path on local computer) and downloads the image file associated with the provided row

In [ ]:
def downloadImage(row,downloadDir):
    bucket = row.s3_bucket.iloc[0]
    key = row.s3_key_image.iloc[0]
    image_name = row.image_name.iloc[0]
    response = s3_client.download_file(bucket,key,downloadDir+image_name)

### Basic Usage Example
This example extracts the first image from the image table and downloads it to a sub directory within the current working directory called "downloads".

In [ ]:
downloadDir = os.path.expanduser('./downloads/')
os.makedirs(downloadDir, exist_ok=True) 

In [ ]:
site_name_4download='4_37' #Select Planet image by UMD GLAD chip number
search = images[images.site_name == site_name_4download] 
search.head()

In [ ]:
row_indx=search[search['site_name'] == site_name_4download].index[0]
row = images.iloc[[row_indx]] 

In [ ]:
downloadImage(row,downloadDir)